In [0]:
# Importando as biliotecas necessárias

import requests
import pandas as pd
from datetime import datetime

In [0]:
# Definição de funções que serão usadas

def extract_bitcoin_price(currency_pair = 'BTC-USD', price_type = 'spot'):

    """Extrai o JSON completo da API da Coinbase com o preço do bitcoin."""
    
    url = f'https://api.coinbase.com/v2/prices/{currency_pair}/{price_type}'
    response = requests.get(url)
    return response.json()


def join_prices(json_dolar_price, json_brl_price, timestamp):

    """A partir dos dados retornados pela API estamos unindo os preços obtidos em real e em dolar 
    apenas 1 objeto, juntamente com o timestamp"""

    # extraindo os preço do json
    dolar_price = float(json_dolar_price['data']['amount'])
    brl_price = float(json_brl_price['data']['amount'])
    cripto_name = 'BTC'

    data = [{
        "dolar_price": dolar_price,
        "brl_price": brl_price,
        "cripto_name": cripto_name,
        "extract_timestamp": timestamp
    }]

    return data

In [0]:
# Gerando as variáveis que serão usadas na funcão de uniao de precos

timestamp = str(datetime.now())
json_dolar_price = extract_bitcoin_price()
json_brl_price = extract_bitcoin_price(currency_pair = 'BTC-BRL')



In [0]:
# gerando objeto com os dados tratados

bitcoin_data = join_prices(json_dolar_price, json_brl_price, timestamp)

In [0]:
print(bitcoin_data)

In [0]:
# Salvando meus dados em um dataframe spark

df_bitcoin_data = spark.createDataFrame(bitcoin_data)

In [0]:
# Variável com o nome da tabela
delta_table_path = "crypto_coins.bitcoin_data.bitcoin_price_hist"

# Salvando dados na tabela delta tabela 
df_bitcoin_data.write \
                .format("delta") \
                .mode("append") \
                .option("mergeSchema", "true") \
                .saveAsTable(delta_table_path)